In [11]:

# Importing necessary libraries
import torch
import numpy as np
import pandas as pd
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

# Define the model name and torch device
model_name = 'tuner007/pegasus_paraphrase'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Initialize the tokenizer and model
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at tuner007/pegasus_paraphrase and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
df = pd.read_csv('converted.csv')
df.head()


,Step,Description
0,1,Initialize quantum registers and classical re...
1,2,Initialize cluster centroids randomly or usin...
2,3,Encode classical data points into quantum states
3,4,Set maximum number of iterations and converge...
4,5,Repeat until convergence or maximum iteration...


In [13]:
#i want to iterate all the elements of df to list
df_list = df[" Description"].values.tolist()
df_list


[' Initialize quantum registers and classical registers',
 ' Initialize cluster centroids randomly or using a deterministic method',
 ' Encode classical data points into quantum states',
 ' Set maximum number of iterations and convergence threshold',
 ' Repeat until convergence or maximum iterations reached',
 ' Assign each data point to the nearest cluster centroid',
 ' Update cluster centroids',
 ' Check for convergence',
 ' Extract final cluster centroids from quantum registers',
 ' Decode quantum cluster centroids to obtain classical representations',
 ' Evaluate clustering quality using metrics such as within-cluster sum of squares',
 ' Output the final cluster centroids and evaluation results']

https://github.com/adarshgowdaa/pegasus-paraphrase/blob/main/main.py

In [14]:
def get_response(input_text, num_return_sequences):
    batch = tokenizer.prepare_seq2seq_batch(
        [input_text], truncation=True, padding='longest', max_length=60, return_tensors="pt").to(torch_device)
    translated = model.generate(**batch, max_length=60, num_beams=10,
                                num_return_sequences=num_return_sequences, temperature=1.5)
    tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
    return tgt_text

In [15]:
#creaye a 2d array of size df list with 10 columns
response = np.empty((len(df_list), 10), dtype=object)

idx = 0
for i in df_list:
    context = i
    num_return_sequences = 10
    num_beams = 10
    response[idx]=get_response(context, num_beams)
    idx = idx + 1

#get_response(context, num_return_sequences)
#context = "Initialize quantum registers to represent data points and cluster centroids."

#val = get_response(context, num_return_sequences)

response

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:4047: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and targets.

Here is a short example:

model_inputs = tokenizer(src_texts, text_target=tgt_texts, ...)

If you either need to use different keyword arguments for the source and target texts, you should do two calls like
this:

model_inputs = tokenizer(src_texts, ...)
labels = tokenizer(text_target=tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/transfor

array([['Initialize classical and quantum registers.',
        'Contribute quantum and classical registers.',
        'Initialize classical and quantum register.',
        'Contribute classical and quantum registers.',
        'Classical and quantum registers should be created.',
        'Contribute quantum and classical register.',
        'The classical and quantum register should be created.',
        'The quantum and classical registers should be created.',
        'The classical and quantum registers should be created.',
        'The quantum and classical register should be created.'],
       ['Use a deterministic method to create cluster centroids.',
        'Use a deterministic method to set up cluster centroids.',
        'cluster centroids can be set randomly or using a deterministic method.',
        'cluster centroids can be used using a deterministic method.',
        'cluster centroids can be randomly or using a deterministic method.',
        'cluster centroids can be ran

In [16]:
#convert response to csv file
df_response = pd.DataFrame(response)
df_response.to_csv('response.csv', index=False)




In [17]:
val



NameError: name 'val' is not defined

In [ ]:
#lavenshtein distance between two strings   
def lev(s, t):
    if s == "":
        return len(t)
    if t == "":
        return len(s)
    if s[-1] == t[-1]:
        cost = 0
    else:
        cost = 1
       
    res = min([lev(s[:-1], t)+1,
               lev(s, t[:-1])+1, 
               lev(s[:-1], t[:-1]) + cost])
    

    
    return res



In [ ]:

def similarity_score(s1, s2):
    value = editdistance.eval(s1, s2)
    return 1 - value / max(len(s1), len(s2))

In [ ]:
similarity_score(context,val[0])
len(val)
   

NameError: name 'editdistance' is not defined

In [ ]:
sim = []

for i in range(len(val)):
    #create an aaray to store the index and similarity score
  
    sim.append(similarity_score(context,val[i]))
print(sim)
#convert sim to 2d array with index and similarity score and add index

sim = np.array(sim)
sim = sim.reshape(-1,1)
sim = np.hstack((sim, np.arange(len(sim)).reshape(-1,1)))
sim = sim[sim[:,0].argsort()[::-1]]
sim

[0.26315789473684215, 0.26315789473684215, 0.2763157894736842, 0.25, 0.25, 0.25, 0.3157894736842105, 0.23684210526315785, 0.3157894736842105, 0.23684210526315785]


array([[0.31578947, 8.        ],
       [0.31578947, 6.        ],
       [0.27631579, 2.        ],
       [0.26315789, 1.        ],
       [0.26315789, 0.        ],
       [0.25      , 5.        ],
       [0.25      , 4.        ],
       [0.25      , 3.        ],
       [0.23684211, 9.        ],
       [0.23684211, 7.        ]])

In [ ]:
#Select the similarity score greater than 0.5
sim = sim[sim[:,0]>0.9]
sim

array([], shape=(0, 2), dtype=float64)